Firedrake implementation of acoustic advection equations: <br>
    $$u_t + U u_x + c_s p_x = 0$$ <br>
    $$p_t + U p_x + c_s u_x = 0$$ <br>
At first we need to find weak formulation which should not require any partial integrations, since only first order derivatives appear. So my guess is, that it is sufficient to just use <br>
    $$\int_\Omega < \left[ \begin{pmatrix}u_t \\ p_t\end{pmatrix} v + \begin{pmatrix} U & 0 \\ 0 & U\end{pmatrix} \begin{pmatrix}u_x \\ p_x\end{pmatrix}v + \begin{pmatrix} 0 & c_s \\ c_s & 0\end{pmatrix} \begin{pmatrix}u_x \\ p_x\end{pmatrix} v\right]> \, \text{d}x $$ <br>
    
BUT: This now only does advection... <br>
$$\psi_t + u \nabla \psi = 0$$

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from firedrake import *
c_s = 1 # speed of sound
U = 0.05 # windspeed
n = 20 # spatial nodes
u_adv=Constant(2.) #advection speed
mesh = PeriodicIntervalMesh(n,1) # create periodic grid
x = SpatialCoordinate(mesh)[0]  # extract coordinates
u_init = sin(2*pi*x)  # create initial conditions using coordinates
V = FunctionSpace(mesh, "Lagrange", 2) # create function Space
u_n1 = Function(V, name="u^{n+1}") 
u_n = Function(V, name="u^{n}")
v = TestFunction(V)
u_n.interpolate(u_init)
dt = 1.0 / n

Use Crank-Nicolson for integration

In [2]:
F = ((u_n1 -u_n)/dt*v+1./2.*((u_adv*u_n1.dx(0)*v)+(u_adv*u_n.dx(0)*v)))*dx

In [3]:
# If passed an existing Function object, the Function 
# constructor makes a copy.
results = [Function(u_n)]

In [10]:
problem = NonlinearVariationalProblem(F, u_n1)
solver = NonlinearVariationalSolver(problem, solver_parameters={"ksp_type": "preonly",
                                                                "pc_type": "lu"})

In [5]:
t = 0
t_end = 0.5
while t <= t_end:
    solver.solve()
    u_n.assign(u_n1)
    results.append(Function(u_n))
    t += dt

In [6]:
# NBVAL_IGNORE_OUTPUT
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_ylim((-1., 1.))
plot(results[0], axes=axes)

<IPython.core.display.Javascript object>

In [7]:
# NBVAL_IGNORE_OUTPUT
from matplotlib.animation import FuncAnimation

def animate(u):
    axes.clear()
    firedrake.plot(u, axes=axes)
    axes.set_ylim((-1., 1.))

interval = 4e3 * float(dt)
animation = FuncAnimation(fig, animate, frames=results, interval=interval)

from IPython.display import HTML
HTML(animation.to_jshtml())

Now proceed with acoustic advection: <br>
Declare variables:

In [128]:
u_n1 = Function(V, name="u^{n+1}") 
u_n = Function(V, name="u^{n}")
p_n1 = Function(V,name="u^{n+1}")
p_n = Function(V, name="u^{n}")
v = TestFunction(V)

Define Fs:

Help on function as_matrix in module ufl.tensors:

as_matrix(expressions, indices=None)
    UFL operator: As *as_tensor()*, but limited to rank 2 tensors.



In [149]:
x_n = as_vector([u_n,p_n])
x_n1 = as_vector([u_n1,p_n1])
mat = as_matrix([[U,c_s],[c_s,U]])

In [ ]:
F1 = ((u_n1-u_n)/dt*v + U*u_n1.dx(0)*v + c_s*p_n1)*dx 

In [156]:
help(as_tensor)

Help on function as_tensor in module ufl.tensors:

as_tensor(expressions, indices=None)
    UFL operator: Make a tensor valued expression.
    
    This works in two different ways, by using indices or lists.
    
    1) Returns :math:`A` such that :math:`A` [*indices*] = *expressions*.
    If *indices* are provided, *expressions* must be a scalar
    valued expression with all the provided indices among
    its free indices. This operator will then map each of these
    indices to a tensor axis, thereby making a tensor valued
    expression from a scalar valued expression with free indices.
    
    2) Returns :math:`A` such that :math:`A[k,...]` = *expressions*[k].
    If no indices are provided, *expressions* must be a list
    or tuple of expressions. The expressions can also consist
    of recursively nested lists to build higher rank tensors.



In [1]:
mat = as_matrix([[1,1],
                 [1,1]])
vec = as_vector([2,3])

# Multiplication operator is overloaded:
mat_vec = mat*vec



NameError: name 'as_matrix' is not defined

In [183]:
indices = [0,1]
len(indices)
mat_vec.evaluate()

TypeError: evaluate() missing 4 required positional arguments: 'x', 'mapping', 'component', and 'index_values'